In [1]:
import pyodbc
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import style
style.use('fivethirtyeight')

# QUERY 01: OPERATIONS

In [2]:
# Connect to the Database
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

In [3]:
df = pd.read_sql_query("DECLARE @from date, @to date;\
    SELECT\
        @from = getdate()\
        ,@to = dateadd(day, 1, getdate());\
\
SELECT CONVERT(varchar,t.completedDateTime,100) Date, w.name warehouse, p.name project, oc.name operationCode, t.employee\
    FROM datex_footprint.Tasks t\
    JOIN datex_footprint.OperationCodes oc\
        ON oc.id = t.operationCodeId\
    JOIN datex_footprint.Warehouses w\
        ON w.id = t.warehouseId\
    JOIN datex_footprint.Projects p\
        ON p.id = t.projectId\
    WHERE t.completedDateTime >= @from AND t.completedDateTime < @to AND (t.employee != 'FootPrintService') AND (t.employee != 'Foot Print Service') and t.statusId = 2", cnxn)

In [4]:
# Adding Hours
df['Hour'] = pd.to_datetime(df['Date']).dt.hour
df.head()

,Date,warehouse,project,operationCode,employee,Hour
0,May 9 2022 1:31PM,20,Alumier Labs Inc.,Receiving,Brittany Owen,13
1,May 9 2022 1:28PM,20,Alumier Labs Inc.,Receiving,Brittany Owen,13
2,May 9 2022 1:28PM,20,Alumier Labs Inc.,Receiving,Brittany Owen,13
3,May 9 2022 1:28PM,20,Alumier Labs Inc.,Receiving,Brittany Owen,13
4,May 9 2022 1:28PM,20,Alumier Labs Inc.,Receiving,Brittany Owen,13


# PICKING

In [5]:
# Filtered by Picking
df_picking = df[df['operationCode'] == 'Picking']

### Picking By Warehouse Function

In [6]:
# Picking Function
def picking_by_warehouse (warehouse):
    # Filter by warehouse
    df_w = df_picking[df_picking['warehouse'] == warehouse]
    
    # Grouped by Employee and Hour
    df_w_group = df_w.groupby(['employee', 'Hour']).count()[['operationCode']]
    df_w_group.reset_index(inplace=True)
    
    # Pivot
    df_w_pivot = df_w_group.pivot(index='employee', columns='Hour', values='operationCode')
    df_w_pivot = df_w_pivot.fillna(0)
    df_w_pivot = df_w_pivot.rename_axis(None, axis=1)
    
    # Save into JSON
    pickers_df = df_w_pivot.T
    file_name = f'data/pickers_w{warehouse}.json'
    pickers_df.to_json(file_name)

In [7]:
# Save warehouse data into JSON files

picking_by_warehouse('10')
#picking_by_warehouse('12')
#picking_by_warehouse('15')
#picking_by_warehouse('16')
#picking_by_warehouse('18')
#picking_by_warehouse('19')
#picking_by_warehouse('20')
#picking_by_warehouse('21')
#picking_by_warehouse('22')

# BATCHING

In [8]:
# Filtered by Batch Move
df_batching = df[df['operationCode'] == 'Batch Move']
df_batching.head()

,Date,warehouse,project,operationCode,employee,Hour
1411,May 9 2022 2:08PM,10,ISDIN Corporation,Batch Move,Rita Geldres,14
1412,May 9 2022 2:08PM,10,ISDIN Corporation,Batch Move,Rita Geldres,14
1413,May 9 2022 2:11PM,10,ISDIN Corporation,Batch Move,Rita Geldres,14
1414,May 9 2022 12:32PM,10,ISDIN Corporation,Batch Move,Liliana Puentes,12
1415,May 9 2022 12:32PM,10,ISDIN Corporation,Batch Move,Liliana Puentes,12


In [9]:
# Batching Function
def batching_by_warehouse (warehouse):
    # Filter by warehouse
    df = df_batching[df_batching['warehouse'] == warehouse]
    
    # Grouped by Employee and Hour
    df = df.groupby(['employee', 'Hour']).count()[['operationCode']]
    df.reset_index(inplace=True)
    
    # Pivot
    df = df.pivot(index='employee', columns='Hour', values='operationCode')
    df = df.fillna(0)
    df = df.rename_axis(None, axis=1)
    
    # Save into JSON
    batchers_df = df.T
    file_name = f'data/batchers_w{warehouse}.json'
    batchers_df.to_json(file_name)

In [10]:
batching_by_warehouse('10')

In [11]:
# Close connection
cnxn.close()

# QUERY: PACKING

In [12]:
# Connect to the Database
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

In [13]:
df_packing = pd.read_sql_query("DECLARE @from date, @to date;\
    SELECT @from = getdate() ,@to = dateadd(day, 1, getdate());\
\
    select t.completedDateTime Date, w.name Warehouse, p.name Project,\
        oc.name OperationCode, t.employee, s.lookupCode Shipment, lp.lookupCode LP\
    from datex_footprint.Tasks t\
        join datex_footprint.Shipments s\
            on s.id = t.shipmentId\
        join datex_footprint.LicensePlates lp\
            on lp.shipmentId = s.id\
        join datex_footprint.Warehouses w\
            on w.id = t.warehouseId\
        join datex_footprint.Projects p\
            on p.id = t.projectId\
        join datex_footprint.OperationCodes oc\
            on oc.id = t.operationCodeId\
        where t.completedDateTime >= @from AND t.completedDateTime < @to AND (t.employee != 'FootPrintService') AND (t.employee != 'Foot Print Service') and t.statusId = 2 and t.operationCodeId = 2067", cnxn)

In [14]:
# Adding Hours
df_packing['Hour'] = pd.to_datetime(df_packing['Date']).dt.hour
df_packing.head()

,Date,Warehouse,Project,OperationCode,employee,Shipment,LP,Hour
0,2022-05-09 11:34:20.747,10,ISDIN Corporation,Rate And Ship,Jackson Pierre,0085647797,OB968198,11
1,2022-05-09 11:32:14.280,10,ISDIN Corporation,Rate And Ship,Jackson Pierre,0085647813,OB968196,11
2,2022-05-09 11:38:44.783,10,ISDIN Corporation,Rate And Ship,Jackson Pierre,0085647932,OB968199,11
3,2022-05-09 11:31:35.290,10,ISDIN Corporation,Rate And Ship,Jackson Pierre,0085647949,OB968197,11
4,2022-05-09 11:33:35.190,10,ISDIN Corporation,Rate And Ship,Jackson Pierre,0085649973,OB968105,11


In [15]:
# Packing Function
def packing_by_warehouse (warehouse):
    # Filter by warehouse
    df_p = df_packing[df_packing['Warehouse'] == warehouse]
    
    # Grouped by Employee and Hour
    df_p_group = df_p.groupby(['employee', 'Hour']).count()[['OperationCode']]
    df_p_group.reset_index(inplace=True)
    
    # Pivot
    df_p_pivot = df_p_group.pivot(index='employee', columns='Hour', values='OperationCode')
    df_p_pivot = df_p_pivot.fillna(0)
    df_p_pivot = df_p_pivot.rename_axis(None, axis=1)
    
    # Save into JSON
    packers_df = df_p_pivot.T
    file_name = f'data/packers_w{warehouse}.json'
    packers_df.to_json(file_name)

In [16]:
# Save warehouse data into JSON files
packing_by_warehouse('10')

In [17]:
# Close connection
cnxn.close()

# QUERY 02: Open Outbounds WAREHOUSE 10

In [18]:
# Connect to the Database

# cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
#                         "Server=wd02;"
#                         "Database=FootPrint;"
#                         "Trusted_Connection=yes;")

In [19]:
# df_02 = pd.read_sql_query("SELECT DISTINCT w.name warehouse, p.name project, s.lookupCode shipment_number, s.createdSysDateTime\
#     FROM datex_footprint.shipments s\
#     LEFT JOIN datex_footprint.ShipmentOrderLookup sol\
#         on sol.shipmentId = s.id\
#     LEFT JOIN datex_footprint.orders o\
#         on o.id = sol.orderId\
#     LEFT JOIN datex_footprint.Projects p\
#         on p.id = o.projectId\
#     LEFT JOIN datex_footprint.warehouses w\
#         on w.id = isnull(s.actualWarehouseId, s.expectedWarehouseId)\
#     WHERE s.statusId in (1, 2, 4) and s.typeId = 2\
#     ORDER BY createdSysDateTime", cnxn)

In [20]:
# df_02.head()

In [21]:
# df_02['age_order'] = pd.Timestamp.now() - df_02['createdSysDateTime']
# df_02.head()

In [22]:
# df_02['age_order_days'] = df_02['age_order'].dt.components['days']
# df_02['age_order_hours'] = df_02['age_order'].dt.components['hours']
# df_02.head()

In [23]:
# Open Orders less than 24 hours

# df_02 = df_02.loc[(df_02['age_order_days'] == 0) & (df_02['warehouse'] == '10')]
# df_02.head()

In [24]:
# Bins per hour

# bins =[-1,4,8,12,16,24]
# bin_names = ['4 Hours', '8 Hours', '12 Hours', '16 Hours', '24 Hours']

# df_02['age_group'] = pd.cut(df_02['age_order_hours'], bins, labels=bin_names)

In [25]:
# df_02_grouped = df_02.groupby(['age_group'])[['project']].count()

In [26]:
# df_02_grouped = df_02_grouped.rename(columns={'project': 'orders'})

### Save in JSON File

In [27]:
# open_outbounds = df_02_grouped
# open_outbounds.to_json('data/open_outbounds.json')
# open_outbounds

In [28]:
# df_02_grouped = df_02_grouped.iloc[::-1]
# df_02_grouped.plot.barh()

In [29]:
# Close connection

# cnxn.close()

# QUERY 3: Picking Performance (Orders and Waves)

In [30]:
# Connect to the Database

# cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
#                         "Server=wd02;"
#                         "Database=FootPrint;"
#                         "Trusted_Connection=yes;")

In [31]:
# df03 = pd.read_sql_query("DECLARE @from date, @to date;\
#                           SELECT @from = getdate(), @to = dateadd(day, 1, getdate());\
#     SELECT t.completedDateTime Date, oc.name Operation, w.name Warehouse,\
#            p.name Project, m.lookupCode Material, employee Employee, s.lookupCode Shipment, isnull(s.waveId, ps.waveId) Wave,\
#            ordersPerWave.shipmentsCount, lc.name Location\
#     FROM datex_footprint.Tasks t\
#         join datex_footprint.OperationCodes oc\
#             on oc.id = t.operationCodeId\
#         left join datex_footprint.Warehouses w\
#             on w.id = t.warehouseId\
#         left join datex_footprint.Projects p\
#             on p.id = t.projectId\
#         left join datex_footprint.Materials m\
#             on m.id = t.materialId\
#         left join datex_footprint.Shipments s\
#             on s.id = t.shipmentId\
#         left join datex_footprint.PickSlips ps\
#             on ps.id = t.pickSlipId\
#         left join datex_footprint.LocationContainers lc\
#             on lc.id = t.actualSourceLocationId\
#         outer apply (\
#             select\
#                 count(distinct tt.shipmentId) shipmentsCount\
#                 from datex_footprint.waves ww\
#                     inner join datex_footprint.PickSlips ppss\
#                     inner join datex_footprint.tasks tt\
#                         on tt.operationCodeId = 8 and tt.pickSlipId = ppss.id\
#                         on ppss.waveId = ww.id\
#                 where ww.id = isnull(s.waveId, ps.waveId)) ordersPerWave\
#     where t.completedDateTime >= @from and t.completedDateTime < @to and t.statusId = 2\
#     and t.operationCodeId in (8, 39, 2064, 2065, 2066) and t.employee != 'Foot Print Service'\
#     order by s.lookupCode, t.completedDateTime", cnxn)

In [32]:
# df03.tail(3)

In [33]:
# Order Columns

# df03 = df03[['Warehouse', 'Project', 'Wave', 'Shipment', 'Date', 'Operation', 'Employee',
#              'Location', 'shipmentsCount', 'Material']]

### Filter by Shipments

In [34]:
# Filter by Orders only

# df03_orders = df03.loc[(df03['Operation']=='Picking') |
#                        (df03['Operation']=='Scanned Shipment') |
#                        (df03['Operation']=='Picking Completed')]

# df03_orders = df03_orders[['Warehouse', 'Project', 'Shipment', 'Date', 'Operation', 'Employee', 'Location']]

# df03_orders.head()

In [35]:
# Create a new column with the time difference between rows

# df03_orders['PickingTime'] = df03_orders['Date'].diff()
# df03_orders.head()

In [36]:
# Add Hour, Minute and Second columns

# df03_orders['hour'] = df03_orders['PickingTime'].dt.components['hours']
# df03_orders['minute'] = df03_orders['PickingTime'].dt.components['minutes']
# df03_orders['second'] = df03_orders['PickingTime'].dt.components['seconds']
# df03_orders.head()

In [37]:
# Set all the Scanned Shipment hour, minute, time to ZERO

# df03_orders_modified = df03_orders

# df03_orders_modified.loc[df03_orders_modified.Operation == 'Scanned Shipment', 'hour'] = 0.0
# df03_orders_modified.loc[df03_orders_modified.Operation == 'Scanned Shipment', 'minute'] = 0.0
# df03_orders_modified.loc[df03_orders_modified.Operation == 'Scanned Shipment', 'second'] = 0.0

# df03_orders_modified.loc[df03_orders_modified['Operation'] == 'Scanned Shipment'].head()

In [38]:
# df03_orders_modified['Time'] = df03_orders_modified['hour']*60 + df03_orders_modified['minute'] + df03_orders_modified['second']/60
# df03_orders_modified.head()

In [39]:
# Save in JSON Format for HTML

# df03_group = df03_orders_modified.groupby(['Warehouse', 'Project', 'Shipment', 'Operation', 'Date'])[['Time']].sum().sort_values(['Warehouse', 'Project', 'Shipment', 'Date'])
# df03_group.to_json('data/performance.json', orient='table')
# df03_group.head(20)

In [40]:
# Remove Operation 'Scanned Shipment'

# df03_excluding_Scanned = df03_orders_modified.loc[df03_orders_modified['Operation'] != 'Scanned Shipment']
# df03_excluding_Scanned.head()

In [41]:
# Save in CSV Format for Tableau

# df03_group2 = df03_excluding_Scanned.groupby(['Warehouse', 'Project', 'Employee', 'Shipment', 'Operation', 'Date'])[['Time']].sum().sort_values(['Warehouse', 'Project', 'Employee', 'Shipment', 'Date'])
# df03_group2.to_csv('data/performance.csv')
# df03_group2.head(20)

In [42]:
# Close connection

# cnxn.close()